# 실루엣계수

In [ ]:
ports = pd.read_csv('./dataset/port.csv')
ports['PORT_CD'].unique()
remove_ports = ['CNSHK','CNTAS','CNYTG','CNYTN',
                    'JPFKY', 'JPHIC', 'JPKNZ', 'JPKOM', 'JPMIZ', 'JPNAO', 'JPONA', 'JPSBS', 'JPSDJ', 'JPSHS', 'JPSKT', 'JPSMN', 'JPTOS',
                    'JPTXD','KRBNP','KRKCN','KRKUV','KRMAS','KRPUS', 'RUVYP','KRUSN',
                    'SGSIN','TWTPE','VNCLI','VNSGN']
print(len(remove_ports))
filtered_ports = ports[~ports['PORT_CD'].isin(remove_ports)]
#filtered_ports.to_csv('../dataset/FINAL/ports_final.csv', index=False)
print(filtered_ports.shape)

# 실루엣 계수를 사용하여 최적의 k 찾기 -> 유클리드 거리 기반
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

ports = pd.read_csv('./dataset/ports_final.csv')
print(ports.shape)
map_center = [35.1, 129.04]  # 부산항

X = ports[['LOC_LAT', 'LOC_LON']] # 부산항이 포함되어 있지 않음.

silhouette_scores = []
k_values = list(range(2, 11))

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)

# 시각화
plt.figure(figsize=(8, 5))
plt.plot(k_values, silhouette_scores, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Optimal k using Silhouette Score (excluding Busan)')
plt.grid(True)
plt.show()
# k=9 or 8 -> 군집 수가 너무 많으면 복잡해질 수도 있으므로 예측 정확도나 군집별 의미 분석 등을 통해 선택

pd.set_option('display.max_colwidth', None)
# 실루엣 계수가 높은 군집 수 결정하여 KMeans
# k = 8
from sklearn.cluster import KMeans

ports = pd.read_csv('./dataset/ports_final.csv')

# KMeans 클러스터링 (k=8)
X = ports[['LOC_LAT', 'LOC_LON']]

kmeans = KMeans(n_clusters=7, random_state=42)
labels = kmeans.fit_predict(X)

# 클러스터 결과 저장
ports['CLUSTER'] = labels

# 군집별 항구 목록을 리스트로 정리
cluster_ports = ports.groupby('CLUSTER')['PORT_CD'].apply(list).reset_index()

# 항구 개수 컬럼 추가
cluster_ports['PORT_COUNT'] = cluster_ports['PORT_CD'].apply(len)

# 컬럼 순서 정리
cluster_ports = cluster_ports.rename(columns={'PORT_CD': 'PORT_LIST'})
cluster_ports = cluster_ports[['CLUSTER', 'PORT_COUNT', 'PORT_LIST']]
cluster_ports

import folium

# 지도 생성
map_center = [35.1, 129.04]
m = folium.Map(location=map_center, zoom_start=4)

# 색상 리스트 (군집 수만큼 필요)
colors = [
    'red', 'blue', 'green', 'purple', 'orange',
    'brown', 'white', 'yellow', 'pink', 'gray'
]

# 마커 표시 (클러스터 없이, 모든 항구 개별 표시)
for idx, row in ports.iterrows():
    cluster = row['CLUSTER']
    popup_text = f"PORT: {row['PORT_CD']}<br>CLUSTER: {cluster}"

    folium.CircleMarker(
    location=[row['LOC_LAT'], row['LOC_LON']],
    radius=6,
    color=colors[cluster % len(colors)],       # 테두리 색
    fill=True,
    fill_opacity=0.7,
    popup=popup_text
).add_to(m)

# 저장
m.save('./dataset/cluster_eight_map.html')
m